In [5]:
#Test on and image
#QAI_HUB mediapipe handlandmark detection
import cv2 as cv
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
import argparse
import numpy as np
from PIL import Image
from qai_hub_models.models.mediapipe_hand.app import MediaPipeHandApp
import math
import pandas as pd

In [6]:
from qai_hub_models.models.mediapipe_hand.model import (
    MODEL_ASSET_VERSION,
    MODEL_ID,
    MediaPipeHand,
)

app = MediaPipeHandApp(MediaPipeHand.from_pretrained(), 0.8, 0.45)

HAND_DETECT_WIDTH = 256
HAND_DETECT_HEIGHT = 256

def preprocess_for_hand_landmark_detect(frame):
    # Resize image
    #images = cv2.cvtColor(frame, c.COLOR_BGR2RGBv2)
    resized_frame = cv.resize(frame, (HAND_DETECT_WIDTH, HAND_DETECT_HEIGHT))
    # Change image to RGB
    #rgb_frame = cv.cvtColor(resized_frame, cv.COLOR_BGR2RGB)
    #Normalize the pixel values to the range [0, 1]
    normalized_frame = resized_frame / 255.0
    normalized_frame = normalized_frame.transpose(2, 0, 1)
    input_tensor = normalized_frame[np.newaxis, :, :, :].astype(np.float32)
    #input_tensor = tf.expand_dims(rgb_frame, axis=0)

    return input_tensor, resized_frame

def preprocess_img(img):
    # fit the image into a 256x256 square
    shape = np.r_[img.shape]
    pad = (shape.max() - shape[:2]).astype('uint32') // 2
    img_pad = np.pad(
        img,((pad[0], pad[0]), (pad[1], pad[1]), (0, 0)),
        mode='constant')
    img_small = cv.resize(img_pad, (256, 256))
    img_small = np.ascontiguousarray(img_small)
    img_norm = np.ascontiguousarray(2 * ((img_small / 255) - 0.5).astype('float32'))
    return img_pad, img_norm, pad


def get_normalized_land_mark_connections(landmarks, bounding_box, x_box_centre, y_box_centre, theta, normalized_edge):
    nomalized_land_mark_pixels = []
    M = cv.getRotationMatrix2D((x_box_centre, x_box_centre), theta, 1.0)
    #Top left bounding box transformed coordinates 
    x_bb_box_tf, y_x_bb_box_tf = bounding_box[0][0][0] #Extracting the top left point
    point = np.array([x_bb_box_tf, y_x_bb_box_tf, 1])
    test = M * point
    x_offset = test[0][0] + test[0][1] + test[0][2]
    y_offset = test[1][0] + test[1][1] + test[1][2]
    
    for ldm in landmarks[0][0]:
        x , y = ldm[:2]
        point = np.array([x, y, 1])
        rotated_point = M * point
        x_ldm_rotated = rotated_point[0][0] + rotated_point[0][1] + rotated_point[0][2]
        y_ldm_rotated = rotated_point[1][0] + rotated_point[1][1] + rotated_point[1][2]

        #Remove the bounding box offset so that we make the pixel point meaningful
        x_ldm_rotated = x_ldm_rotated - x_offset
        y_ldm_rotated = y_ldm_rotated - y_offset
        
        nomalized_land_mark_pixels.append(x_ldm_rotated / normalized_edge)
        nomalized_land_mark_pixels.append(y_ldm_rotated / normalized_edge)

    #After all the land mark points are input we input the theta
    nomalized_land_mark_pixels.append(theta)
    
    '''
    for connection in connections:
        x1, y1 = (land_mark_points[connection[0]])
        x2, y2 = (land_mark_points[connection[1]])
        edge_length = math.sqrt((int(x2) - int(x1)) ** 2 + (int(y2) - int(y1)) ** 2)
        if bounding_box_edge != 0:
            nomalized_land_mark_connections.append(edge_length/bounding_box_edge)
        #nomalized_land_mark_connections.append
        else:
            print("Bouding box size is 0")
    '''
    
    return nomalized_land_mark_pixels

def bouding_box_details(bounding_box):
    x1, y1 = bounding_box[0][0][0] #Extracting the top left point
    x2, y2 = bounding_box[0][0][2] #Extracting the top right point
    x3, y3 = bounding_box[0][0][1] #Extracting the bottom left point
    x4, y4 = bounding_box[0][0][1] #Extracting the bottom right point
   
    x_mid_top_line = (int(x2) + int(x1))/2
    y_mid_top_line = (int(y2) + int(y1))/2

    x_mid_left_side_line = (int(x3) + int(x1))/2
    y_mid_left_side_line = (int(y3) + int(y1))/2

    top_edge = math.sqrt((int(x2) - int(x1)) ** 2 + (int(y2) - int(y1)) ** 2)
    side_edge = math.sqrt((int(x3) - int(x1)) ** 2 + (int(y3) - int(y1)) ** 2)
    normalized_edge = math.sqrt(top_edge ** 2 + side_edge ** 2)
    
    if(int(x2) != int(x1)):
        slope_top_line = (int(y2) - int(y1))/(int(x2) - int(x1))
        theta = int(np.arctan(slope_top_line) * 57.29) #converting radian to theta
        
    else:
        theta = 90
        x_box_centre = x_mid_top_line
        y_box_centre = y_mid_left_side_line
        return x_box_centre, y_box_centre, theta, normalized_edge
        
    if(int(x3) != int(x1)):
        slope_left_line = (int(y3) - int(y1))/(int(x3) - int(x1))
    else:
        theta = 90
        x_box_centre = x_mid_top_line
        y_box_centre = y_mid_left_side_line
        return x_box_centre, y_box_centre, theta, normalized_edge

    slope_perpendicular_top_line = -1 * slope_top_line
    slope_perpendicular_left_line = -1 * slope_left_line
    
    a1 = slope_perpendicular_top_line
    b1 = -1
    c1 = y_mid_top_line - (slope_perpendicular_top_line * x_mid_top_line)

    a2 = slope_perpendicular_left_line
    b2 = -1
    c2 = y_mid_left_side_line - (slope_perpendicular_left_line * x_mid_left_side_line)

    x_box_centre = ((b1 * c2) - (b2 * c1))/((a1 * b2) - (a2 * b1))
    y_box_centre = ((a2 * c1) - (a1 * c2))/((a1 * b2) - (a2 * b1))
    
    return x_box_centre, y_box_centre, theta, normalized_edge



In [7]:
from tensorflow.keras.models import load_model
new_model = load_model(r"C:\Users\atandebn\Downloads\models\hand_landmark_to_sign_large_data.h5")
new_model.summary()

Model: "sequential_43"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_130 (Dense)                    │ (None, 128)                 │           5,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_131 (Dense)                    │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_132 (Dense)                    │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_133 (Dense)                    │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_134 (Dense)                    │ (None, 29)                  │           3,741 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 58,911 (230.12 KB)

 Trainable params: 58,909 (230.11 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [31]:
import time
import string

cam = cv.VideoCapture(0)
#frame_width = int(cam.get(cv.CAP_PROP_FRAME_WIDTH))
#frame_height = int(cam.get(cv.CAP_PROP_FRAME_HEIGHT))

character_list = list(string.ascii_uppercase)
decoded_character_list = []
conversation_string = []
frame_rate = 10
prev = 0

def most_frequent(List):
    counter = 0
    num = List[0]
    
    for i in List:
        curr_frequency = List.count(i)
        if(curr_frequency> counter):
            counter = curr_frequency
            num = i

    return num

i = 0
repeat = 0
last_char = None
while True:
    time_elapsed = time.time() - prev
    ret, frame = cam.read() #Reading the frames
    
    if time_elapsed > 1/frame_rate:
        cv.imshow('frame', frame)
        prev = time.time()
        img_pad, img_norm, pad = preprocess_img(frame)
        a, b, c, d, e = app.predict_landmarks_from_image(img_pad, raw_output=True)
        if d == [None] or d == [None, None]:
            continue;
        else:   
            x_box_centre, y_box_centre, theta, normalized_edge = bouding_box_details(c)
            nomalized_land_mark_pixels = get_normalized_land_mark_connections(d, c, x_box_centre, y_box_centre, theta, normalized_edge)
            x_test = np.array(nomalized_land_mark_pixels, dtype=np.float32)
            x_test = x_test.reshape(1,43)
            result = new_model.predict(x_test)
            #print(result.shape)
            max_ele_idx = np.argmax(result)
            if result[0, max_ele_idx] < 0.6:
                print("No Letter")
            else:
                i = i + 1
                #print("Decoded charatcter: ", character_list[max_ele_idx])
                decoded_character_list.append(max_ele_idx)
                #print("Decoded character list: ", decoded_character_list)
                if(len(decoded_character_list) >= 5):
                    if(last_char == character_list[most_frequent(decoded_character_list)] and (repeat < 2)):
                        repeat = repeat + 1
                    else:
                        repeat = 0
                        conversation_string = character_list[most_frequent(decoded_character_list)]
                        print("Decoded character:", conversation_string)
                        #__draw_label(frame, character_list[most_frequent(decoded_character_list)], (10,10), (255,0,0))
                        last_char = conversation_string
                        decoded_character_list = []
                            
                if(i >= 5):
                    i = 0
                    decoded_character_list = []
                
                     
    if cv.waitKey(1) == ord('d'):
        break

cam.release()
cv.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
No Letter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
No Letter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
No Letter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
No Letter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
No Letter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
No Letter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
No Letter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
No Letter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
No Letter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
No Letter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
No Letter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Decoded character: G
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
No Letter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
No Letter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
No Letter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
No 